In [3]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import euclidean

# Function to load coordinates from CSV and extract positions for specific atoms
def load_and_extract_coordinates(file_path):
    # Read the CSV file
    coords = pd.read_csv(file_path)
    
    # Print column names for debugging
    print("Columns in CSV:", coords.columns)
    
    # Extracting positions for the relevant atoms (Atom_49, Atom_83, Atom_196)
    # Using the correct column names from the CSV
    atom_49_positions = coords[['Atom_49_x', 'Atom_49_y', 'Atom_49_z']].values
    atom_83_positions = coords[['Atom_83_x', 'Atom_83_y', 'Atom_83_z']].values
    atom_196_positions = coords[['Atom_196_x', 'Atom_196_y', 'Atom_196_z']].values

    # Creating a DataFrame to store the positions
    atom_data = {
        'Atom_49_x': atom_49_positions[:, 0],
        'Atom_49_y': atom_49_positions[:, 1],
        'Atom_49_z': atom_49_positions[:, 2],
        'Atom_83_x': atom_83_positions[:, 0],
        'Atom_83_y': atom_83_positions[:, 1],
        'Atom_83_z': atom_83_positions[:, 2],
        'Atom_196_x': atom_196_positions[:, 0],
        'Atom_196_y': atom_196_positions[:, 1],
        'Atom_196_z': atom_196_positions[:, 2]
    }
    
    # Include the Frame column if it exists
    if 'Frame' in coords.columns:
        atom_data['Frame'] = coords['Frame'].values
        
    atom_df = pd.DataFrame(atom_data)
    
    return atom_df

# Define charges for product state (state 2)
product_charges = {
    '49.SG': -0.9,     # Atom_49 (SG)
    '49.HG1': 0.418,
    '49.CB': -0.3,
    '49.HB1': 0.1,
    '49.HB2': 0.1,
    '49.CA': 0.14,
    '49.HA': 0.06,
    '49.N': -0.5,
    '49.HN': 0.3,
    '196.O1': -0.418,  # Atom_196
    '196.O2': -0.418,
    '196.H1': 0.418,
    '196.H2': 0.418,
    '83.NE2': -0.4965, # Atom_83 (NE2)
    '83.HE21': 0.48,
    '83.HE22': 0.48,
    '83.CD': 0.2858,
    '83.OE1': -0.503,
    '83.CB': -0.12,
    '83.HB1': 0.06,
    '83.HB2': 0.06,
    '83.CG': 0.2157,
    '83.HG1': 0.06,
    '83.HG2': 0.06
}

# Define constant values for the calculations
C = 1 / (4 * np.pi * 8.854e-12)  # Coulomb's constant approximation in vacuum
D = 1  # Defined constant

# File path for Product state
product_file = '/home/hp/results/MOUSE/level1/D148E/replica000/product.csv'

# Load and extract coordinates for Product state
product_atom_data = load_and_extract_coordinates(product_file)

# Calculate center of charges
def calculate_center_of_charges(df, charges):
    # We'll use the positions of the atoms and their charges to calculate the center of charges
    # For simplicity, we'll use the primary atoms for which we have coordinates
    
    # Get positions for key atoms
    atom_49_pos = np.array([df['Atom_49_x'].mean(), df['Atom_49_y'].mean(), df['Atom_49_z'].mean()])
    atom_83_pos = np.array([df['Atom_83_x'].mean(), df['Atom_83_y'].mean(), df['Atom_83_z'].mean()])
    atom_196_pos = np.array([df['Atom_196_x'].mean(), df['Atom_196_y'].mean(), df['Atom_196_z'].mean()])
    
    # Get charges for key atoms
    charge_49 = charges['49.SG']
    charge_83 = charges['83.NE2']
    charge_196 = charges['196.O1'] + charges['196.O2']  # Combined charge for atom 196
    
    # Calculate weighted positions
    weighted_pos = (atom_49_pos * charge_49 + atom_83_pos * charge_83 + atom_196_pos * charge_196)
    total_charge = charge_49 + charge_83 + charge_196
    
    # Calculate center of charges
    center_of_charges = weighted_pos / total_charge if total_charge != 0 else np.zeros(3)
    
    return center_of_charges

# Calculate dipole interactions
def calculate_dipole_interaction(df, charges):
    # Get positions for key atoms
    atom_49_pos = np.array([df['Atom_49_x'].mean(), df['Atom_49_y'].mean(), df['Atom_49_z'].mean()])
    atom_83_pos = np.array([df['Atom_83_x'].mean(), df['Atom_83_y'].mean(), df['Atom_83_z'].mean()])
    atom_196_pos = np.array([df['Atom_196_x'].mean(), df['Atom_196_y'].mean(), df['Atom_196_z'].mean()])
    
    # Get charges for key atoms
    charge_49 = charges['49.SG']
    charge_83 = charges['83.NE2']
    charge_196 = charges['196.O1'] + charges['196.O2']  # Combined charge for atom 196
    
    # Calculate distances between atoms
    dist_49_83 = euclidean(atom_49_pos, atom_83_pos)
    dist_49_196 = euclidean(atom_49_pos, atom_196_pos)
    dist_83_196 = euclidean(atom_83_pos, atom_196_pos)
    
    # Calculate Coulomb interactions (k * q1 * q2 / r)
    # Using the provided Coulomb's constant C
    coulomb_49_83 = (C * charge_49 * charge_83) / dist_49_83
    coulomb_49_196 = (C * charge_49 * charge_196) / dist_49_196
    coulomb_83_196 = (C * charge_83 * charge_196) / dist_83_196
    
    # Total dipole interaction
    total_dipole_interaction = coulomb_49_83 + coulomb_49_196 + coulomb_83_196
    
    # Calculate dipole moments as in the provided code
    dipole_moments = {
        '49-83.NE2': (C * charge_49 * charge_83) / (dist_49_83 ** 2),
        '49-196.O1': (C * charge_49 * charge_196/2) / (dist_49_196 ** 2),
        '49-196.O2': (C * charge_49 * charge_196/2) / (dist_49_196 ** 2)
    }
    
    # Calculate potential using theta = π/4 as in the example
    theta = np.pi / 4
    potential_49_83 = (C * dipole_moments['49-83.NE2'] * charge_49 * np.cos(theta)) / (D * dist_49_83**2)
    potential_49_196 = (C * dipole_moments['49-196.O1'] * charge_49 * np.cos(theta)) / (D * dist_49_196**2)
    
    return {
        'Atom_49_83_interaction': coulomb_49_83,
        'Atom_49_196_interaction': coulomb_49_196,
        'Atom_83_196_interaction': coulomb_83_196,
        'Total_dipole_interaction': total_dipole_interaction,
        'dist_49_83': dist_49_83,
        'dist_49_196': dist_49_196,
        'dist_83_196': dist_83_196,
        'Dipole_49-83.NE2': dipole_moments['49-83.NE2'],
        'Dipole_49-196.O1': dipole_moments['49-196.O1'],
        'Dipole_49-196.O2': dipole_moments['49-196.O2'],
        'Potential_49-83': potential_49_83,
        'Potential_49-196': potential_49_196
    }

# Calculate center of charges
center_of_charges = calculate_center_of_charges(product_atom_data, product_charges)

# Calculate dipole interactions
dipole_interactions = calculate_dipole_interaction(product_atom_data, product_charges)

# Add center of charges to the dataframe
product_atom_data['Center_of_charges_x'] = center_of_charges[0]
product_atom_data['Center_of_charges_y'] = center_of_charges[1]
product_atom_data['Center_of_charges_z'] = center_of_charges[2]

# Add dipole interaction results to the dataframe
for key, value in dipole_interactions.items():
    product_atom_data[key] = value

# Add charges to the dataframe
product_atom_data['Atom_49_charge'] = product_charges['49.SG']
product_atom_data['Atom_83_charge'] = product_charges['83.NE2']
product_atom_data['Atom_196_charge'] = product_charges['196.O1'] + product_charges['196.O2']

# Save the extracted data into a new CSV file
product_atom_data.to_csv('/home/hp/results/MOUSE/level1/D148E/replica000/product_atom_positions_with_charges.csv', index=False)

print("Product atom positions with charges and dipole interactions saved to product_atom_positions_with_charges.csv")
print(f"Center of charges: {center_of_charges}")
print(f"Dipole interactions: {dipole_interactions}")

Columns in CSV: Index(['Frame', 'Atom_49_x', 'Atom_49_y', 'Atom_49_z', 'Atom_83_x',
       'Atom_83_y', 'Atom_83_z', 'Atom_196_x', 'Atom_196_y', 'Atom_196_z'],
      dtype='object')
Product atom positions with charges and dipole interactions saved to product_atom_positions_with_charges.csv
Center of charges: [35.3076965  26.67028727 26.93819844]
Dipole interactions: {'Atom_49_83_interaction': 516741489.1476577, 'Atom_49_196_interaction': 358659790.7795334, 'Atom_83_196_interaction': 187990455.72370192, 'Total_dipole_interaction': 1063391735.650893, 'dist_49_83': 7.772111960739858, 'dist_49_196': 18.85457356578658, 'dist_83_196': 19.84450854350032, 'Dipole_49-83.NE2': 66486624.45393633, 'Dipole_49-196.O1': 9511214.600746945, 'Dipole_49-196.O2': 9511214.600746945, 'Potential_49-83': -6295559282330838.0, 'Potential_49-196': -153031445974804.2}
